In [1]:
import csv
import pickle
import datetime

In [2]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png','retina'}

In [3]:
data = pd.read_csv('data/data.csv')
data=data.iloc[:,1:]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (6,7,8,10,12,13,14,15,18,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
new_order =  ['date', 'store_nbr', 'item_nbr', 'station_nbr', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'avgspeed', 'resultspeed', 'resultdir', 'TS', 'GR', 'RA', 'DZ', 'SN',
       'SG', 'GS', 'PL', 'FG+', 'FG', 'BR', 'UP', 'HZ', 'FU', 'DU', 'SQ', 'FZ',
       'MI', 'PR', 'BC', 'BL', 'VC' , 'units' ]

In [5]:
data = data[new_order]

In [6]:
from sklearn.model_selection import train_test_split
X, y = data.iloc[:,:-1], data.units
X_train, X_testm, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [7]:
def fulldf(df):
    
    df.snowfall = df.snowfall.replace(["T","  T"],0.05)
    df.preciptotal = df.preciptotal.replace(["T","  T"],0.005)
    df.depart = pd.to_numeric(df.depart, errors='coerce')
    
    df = type_change_numeric(df,[ 'store_nbr', 'item_nbr', 'units', 'station_nbr', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'avgspeed', 'resultspeed', 'resultdir' ])
    df['date'] = pd.to_datetime(df['date'])
    df["week_day_name"] = df['date'].dt.weekday_name
    df["day_of_year"] = df['date'].dt.dayofyear
    df["year"] = df['date'].dt.year
    
    
    
    for idx in range(5, 8):
        df.iloc[:,idx].fillna(df.groupby(["day_of_year","store_nbr"])[df.columns[idx]].\
                              transform('mean'), inplace=True)

    for idx in range(16, 23):
        df.iloc[:,idx].fillna(df.groupby(["day_of_year","store_nbr"])[df.columns[idx]].\
                              transform('mean'), inplace=True)
    add_depart1(df)
    
    return df

# for column in item37.columns:
#     item37[column].interpolate()


In [8]:
def itemdf(df, nbr):
    item_nbr = df[df.item_nbr == nbr]
    for idx in range(5, 8):
        item_nbr.iloc[:,idx].fillna(item_nbr.groupby(["day_of_year","store_nbr"])[item_nbr.columns[idx]].\
                          transform('mean'), inplace=True)
    
    for idx in range(16, 23):
        item_nbr.iloc[:,idx].fillna(item_nbr.groupby(["day_of_year","store_nbr"])[item_nbr.columns[idx]].\
                          transform('mean'), inplace=True)
    add_depart1(item_nbr)
    
    return item_nbr
    

In [9]:
def type_change_numeric(df, ls = []): 
    #ls에 있는 column name은 numeric형으로 바꾸지 않는다.
    cols = df.columns
    for i in cols:
        if i in ls:
            #df = df.replace(["M",None], '')
            df.snowfall = df.snowfall.replace(["T","  T"],0.05)
            df.preciptotal = df.preciptotal.replace(["T","  T"],0.005)
            df[i] = pd.to_numeric(df[i], errors='coerce')
        
    return df

#예시

# data = type_change_numeric(data,[ 'store_nbr', 'item_nbr', 'units', 'station_nbr', 'tmax', 'tmin',
#        'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
#        'avgspeed', 'resultspeed', 'resultdir' ])

In [14]:
def add_depart1(x):
    x.depart.fillna(x.tavg - x.groupby(["day_of_year","store_nbr"])["tavg"].transform('mean'),inplace = True)
    x.depart = x.depart.round(2)
    return x.sort_values(["store_nbr","date","item_nbr"])

In [10]:
train = pd.concat([X_train, y_train] ,axis = 1)

In [11]:
train.head()

,date,store_nbr,item_nbr,station_nbr,tmax,tmin,tavg,depart,dewpoint,wetbulb,...,FU,DU,SQ,FZ,MI,PR,BC,BL,VC,units
258351,2012-03-11,39,36,8,70,55,M,M,48,53,...,False,False,False,False,False,False,False,False,False,14.0
96002,2014-10-05,8,59,4,87,62,75,1,62,66,...,False,False,False,False,False,False,False,False,False,1.0
217631,2013-10-02,17,71,20,89,74,82,M,69,72,...,False,False,False,False,False,False,False,False,False,0.0
12167,2012-08-04,6,45,14,109,80,95,12,60,72,...,False,False,False,False,False,False,False,False,False,66.0
203388,2014-07-29,25,9,13,81,57,69,M,55,59,...,False,False,False,False,False,False,False,False,False,0.0


In [12]:
train = train.sort_values(by=['date', 'store_nbr', 'item_nbr']).reset_index(drop= True)

In [15]:
df = fulldf(train)

In [16]:
item37 = df[df.item_nbr == 37]


In [17]:
item37 = item37.sort_values(by=['date', 'store_nbr', 'item_nbr']).reset_index(drop= True)

In [18]:
item37.tmax.fillna(item37.groupby(["day_of_year","store_nbr"])['tmax'].transform('mean'), inplace=True) 

In [19]:
item37 = item37[item37.tmax.isnull() == False]
item37 = item37[item37.tmin.isnull() == False]
item37 = item37[item37.dewpoint.isnull() == False]
item37 = item37[item37.wetbulb.isnull() == False]
item37 = item37[item37.preciptotal.isnull() == False]

In [20]:
item37.isnull().any()

date             False
store_nbr        False
item_nbr         False
station_nbr      False
tmax             False
tmin             False
tavg             False
depart           False
dewpoint         False
wetbulb          False
heat              True
cool              True
sunrise           True
sunset            True
snowfall          True
preciptotal      False
stnpressure      False
sealevel         False
avgspeed         False
resultspeed      False
resultdir        False
TS               False
GR               False
RA               False
DZ               False
SN               False
SG               False
GS               False
PL               False
FG+              False
FG               False
BR               False
UP               False
HZ               False
FU               False
DU               False
SQ               False
FZ               False
MI               False
PR               False
BC               False
BL               False
VC               False
units      

In [ ]:
data.snowfall = data.snowfall.replace(["T","  T"],0.05)
data.preciptotal = data.preciptotal.replace(["T","  T"],0.005)
data.depart = pd.to_numeric(data.depart, errors='coerce')



In [ ]:
data['date'] = pd.to_datetime(data['date'])
#data['sunrise'] = pd.to_datetime(data['sunrise'], format='%H%M' , errors='coerce')
#data['sunset'] = pd.to_datetime(data['sunset'], format='%H%M' , errors='coerce')
#X_testm['date'] = pd.to_datetime(X_testm['date'] , errors='coerce')
#X_testm['sunrise'] = pd.to_datetime(X_testm['sunrise'], format='%H%M' , errors='coerce')
#X_testm['sunset'] = pd.to_datetime(X_testm['sunset'], format='%H%M' , errors='coerce')

In [ ]:
data["week_day_name"] = data.date.dt.weekday_name
data["day_of_year"] = data.date.dt.dayofyear
data["year"] = data.date.dt.year

In [ ]:
#data.tmax.fillna(data.groupby(["day_of_year","store_nbr"])['tmax'].transform('mean'), inplace=True)
data = data.replace(["M",None], '')

In [ ]:
for idx in range(5, 8):
    item37.iloc[:,idx].fillna(item37.groupby(["day_of_year","store_nbr"])[item37.columns[idx]].\
                              transform('mean'), inplace=True)

In [ ]:
for idx in range(16, 23):
    item37.iloc[:,idx].fillna(item37.groupby(["day_of_year","store_nbr"])[item37.columns[idx]].\
                              transform('mean'), inplace=True)

In [21]:
item37['weekend'] = False
item37.weekend[item37['week_day_name'] == 'Sunday'] = True
item37.weekend[item37['week_day_name'] == 'Saturday'] = True
item37.weekend[item37['week_day_name'] == 'Friday'] = True

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [ ]:
item37['reldate'] = 0
item37.reldate[ item37.date.dt.year == 2012] = item37.date.dt.dayofyear
item37.reldate[ item37.date.dt.year != 2012] = item37.date.dt.dayofyear + (item37.date.dt.year- 2012)*365 + 1

In [ ]:
item37.isnull().any()

In [ ]:
item37_df.head()

In [23]:
item37.columns

Index(['date', 'store_nbr', 'item_nbr', 'station_nbr', 'tmax', 'tmin', 'tavg',
       'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise', 'sunset',
       'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'avgspeed',
       'resultspeed', 'resultdir', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS',
       'PL', 'FG+', 'FG', 'BR', 'UP', 'HZ', 'FU', 'DU', 'SQ', 'FZ', 'MI', 'PR',
       'BC', 'BL', 'VC', 'units', 'week_day_name', 'day_of_year', 'year',
       'weekend'],
      dtype='object')

In [24]:
item37.columns =['date', 'store_nbr', 'item_nbr', 'station_nbr', 'tmax', 'tmin', 'tavg',
       'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise', 'sunset',
       'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'avgspeed',
       'resultspeed', 'resultdir', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS',
       'PL', 'FG2', 'FG', 'BR', 'UP', 'HZ', 'FU', 'DU', 'SQ', 'FZ', 'MI', 'PR',
       'BC', 'BL', 'VC', 'units', 'week_day_name', 'day_of_year', 'year',
       'weekend']

#### OLS

In [26]:
model = sm.OLS.from_formula("units ~  C(store_nbr) + tmax + tmin + tavg + depart + dewpoint + wetbulb + snowfall + preciptotal + stnpressure + sealevel + avgspeed + resultspeed + resultdir + C(TS) + C(GR) + C(RA) + C(DZ) + C(SN) + C(SG) + C(GS) + C(PL) + C(FG2) + C(FG) + C(BR) + C(UP) + C(HZ) + C(FU) + C(DU) + C(SQ) + C(FZ) + C(MI) + C(PR) + C(BC) + C(BL) + C(weekend) + 0" , data = item37)
#model = sm.OLS.from_formula("units ~  date + store_nbr + tmax  + tavg + depart + dewpoint + wetbulb + snowfall + preciptotal + stnpressure + sealevel + avgspeed + resultspeed + resultdir + C(TS) + C(GR) + C(RA) + C(DZ) + C(SN) + C(SG) + C(GS) + C(PL) + C(FG2) + C(FG) + C(BR) + C(UP) + C(HZ) + C(FU) + C(DU) + C(SQ) + C(FZ) + C(MI) + C(PR) + C(BC) + C(BL) + C(weekend) + 0" , data = item37_df)

result1 = model.fit()


NameError: name 'item37_df' is not defined

In [30]:
item37['logunits'] = item37.units.apply(lambda X: np.log(X) + 1)

In [32]:
item37.head()

,date,store_nbr,item_nbr,station_nbr,tmax,tmin,tavg,depart,dewpoint,wetbulb,...,PR,BC,BL,VC,units,week_day_name,day_of_year,year,weekend,logunits
0,2012-01-01,5,37,12,72.0,48.0,60.0,0.00,54.0,57.0,...,False,False,False,False,16.0,Sunday,1,2012,True,3.772589
1,2012-01-01,10,37,12,72.0,48.0,60.0,-0.29,54.0,57.0,...,False,False,False,False,32.0,Sunday,1,2012,True,4.465736
2,2012-01-01,12,37,11,72.0,48.0,60.0,7.00,54.0,56.0,...,False,False,False,False,28.0,Sunday,1,2012,True,4.332205
3,2012-01-01,17,37,20,59.0,41.0,50.0,1.33,28.0,40.0,...,False,False,False,False,85.0,Sunday,1,2012,True,5.442651
4,2012-01-02,5,37,12,65.0,47.0,56.0,-4.43,33.0,46.0,...,False,False,False,False,29.0,Monday,2,2012,False,4.367296


In [ ]:
print(result1.summary())

In [28]:
model = sm.OLS.from_formula("units ~  C(store_nbr) + scale(tmax) + scale(tmin) + scale(tavg) + scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(avgspeed) + scale(resultspeed)   + C(weekend) + 0" , data = item37)
model = sm.OLS.from_formula("units ~  date + store_nbr + tmax  + tavg + depart + dewpoint + wetbulb + snowfall + preciptotal + stnpressure + sealevel + avgspeed + resultspeed + resultdir + C(TS) + C(GR) + C(RA) + C(DZ) + C(SN) + C(SG) + C(GS) + C(PL) + C(FG2) + C(FG) + C(BR) + C(UP) + C(HZ) + C(FU) + C(DU) + C(SQ) + C(FZ) + C(MI) + C(PR) + C(BC) + C(BL) + C(weekend) + 0" , data = item37_df)

result1 = model.fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.611
Method:                 Least Squares   F-statistic:                     391.4
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        02:30:22   Log-Likelihood:                -16958.
No. Observations:                4223   AIC:                         3.395e+04
Df Residuals:                    4205   BIC:                         3.407e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(store_nbr)[5]        9.6262      2

In [34]:
pickle.dump( item37, open( "item37.p", "wb" ) )

In [33]:
log_model = sm.OLS.from_formula("logunits ~  C(store_nbr) + scale(tmax) + scale(tmin) + scale(tavg) + scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(avgspeed) + scale(resultspeed)   + C(weekend) + 0" , data = item37)
#model = sm.OLS.from_formula("units ~  date + store_nbr + tmax  + tavg + depart + dewpoint + wetbulb + snowfall + preciptotal + stnpressure + sealevel + avgspeed + resultspeed + resultdir + C(TS) + C(GR) + C(RA) + C(DZ) + C(SN) + C(SG) + C(GS) + C(PL) + C(FG2) + C(FG) + C(BR) + C(UP) + C(HZ) + C(FU) + C(DU) + C(SQ) + C(FZ) + C(MI) + C(PR) + C(BC) + C(BL) + C(weekend) + 0" , data = item37_df)

log_result1 = log_model.fit()
print(log_result1.summary())

                            OLS Regression Results                            
Dep. Variable:               logunits   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 04 Jul 2018   Prob (F-statistic):                nan
Time:                        02:34:07   Log-Likelihood:                    nan
No. Observations:                4223   AIC:                               nan
Df Residuals:                    4205   BIC:                               nan
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(store_nbr)[5]           nan       

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:1404: RuntimeWarning: invalid value encountered in less
  term2 = np.where(denom < 0, term1, np.power((1-2.0/A)/denom, 1/3.0))
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1366: RuntimeWarning: invalid value encountered in subtract
  weights=weights))**2)


In [40]:
model2 = sm.OLS.from_formula("units ~  C(store_nbr) + scale(tmax) + scale(tmin) + scale(tavg) + scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(avgspeed) + scale(resultspeed) + C(TS) + C(GR) + C(RA) + C(DZ) + C(SN) + C(SG) + C(GS) + C(PL) + C(FG2) + C(FG) + C(BR) + C(UP) + C(HZ) + C(FU) + C(DU) + C(SQ) + C(FZ) + C(MI) + C(PR) + C(BC) + C(BL) + C(weekend) + 0" , data = item37)


result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.618
Model:                            OLS   Adj. R-squared:                  0.615
Method:                 Least Squares   F-statistic:                     205.6
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        02:49:16   Log-Likelihood:                -16928.
No. Observations:                4223   AIC:                         3.392e+04
Df Residuals:                    4189   BIC:                         3.414e+04
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(store_nbr)[5]        9.4828      2